In [ ]:
lr = 1e-6
dataset = 'manual'
max_train_steps = 2000

# Installation

In [ ]:
checkpoints_path = 'gdrive/MyDrive/inpainting/checkpoints'
train_data_path = f'gdrive/MyDrive/inpainting/train_data_{dataset}'
# class_data only for prior preservation loss
class_data_path = f'gdrive/MyDrive/inpainting/class_data'
synthetic_data_path = 'gdrive/MyDrive/inpainting/synthetic_data/manual'

In [ ]:
from google.colab import drive
drive.mount('gdrive/')

Mounted at gdrive/


In [ ]:
!pip install git+https://github.com/huggingface/diffusers
!git clone https://github.com/huggingface/diffusers
!pip install -U -r diffusers/examples/research_projects/dreambooth_inpaint/requirements.txt
!pip install accelerate
!pip install bitsandbytes
!pip install xformers

  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-7wpzaz2c
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-7wpzaz2c
  Resolved https://github.com/huggingface/diffusers to commit e46ec5f88fec23870538df782258c59271b010fd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.2 MB/s eta 0:00:00
  Created wheel for diffusers: filename=diffusers-0.22.0.dev0-py3-none-any.whl size=1547466 sha256=1e32c44efda002f05321fe1fb0ab329759515e7c3eae18880b26aec08a2140b3
  Stored in directory: /tmp/pip-ephem-wheel-cache-82oeus87/wheels/f7/7d/99/d361489e5762e3464b3811bc629e94cf5bf5ef44dd5c3c4d52
Successfully built diffusers
Cloning into 'diffusers'...
remote: Enumerating objects: 39129, done.
remote:

In [ ]:
!accelerate config default

accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml


In [ ]:
!pip install -U torch==2.0.0+cu118 torchvision==0.15.1+cu118 --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 796.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 82.7 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1
    Uninstalling torch-2.0.1:
      Successfully uninstalled torch-2.0.1
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.16.0
    Uninstalling torchvision-0.16.0:
      Successfully uninstalled torchvision-0.16.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 2.0.0+cu118 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you have torch 2.0.0+cu118 which is incompatible.
torchtext 0.15.2 requires torch==2.0.1, but you have torch 2.0.0+cu118 which is incompatibl

In [ ]:
!pip install -U diffusers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 14.0 MB/s eta 0:00:00
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.9.0
    Uninstalling diffusers-0.9.0:
      Successfully uninstalled diffusers-0.9.0


In [ ]:
from diffusers import StableDiffusionInpaintPipeline, UNet2DConditionModel
from transformers import CLIPTextModel
import torch
import PIL
from datetime import datetime

# Training

In [ ]:
!accelerate launch diffusers/examples/research_projects/dreambooth_inpaint/train_dreambooth_inpaint.py \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-inpainting"  \
  --instance_data_dir=$train_data_path \
  --output_dir=$checkpoints_path \
  --instance_prompt="satellite imagery, wildfire, landsat-8" \
  --resolution=256 \
  --train_batch_size=2 \
  --gradient_accumulation_steps=2 \
  --gradient_checkpointing \
  --use_8bit_adam \
  --learning_rate=$lr \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=$max_train_steps \
  --checkpointing_steps=3000

2023-08-30 11:24:28.768250: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-30 11:24:35.572847: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Steps: 100% 2000/2000 [1:05:19<00:00,  1.96s/it, loss=0.00359, lr=1e-6]
safety_checker/model.safetensors not found

Fetching 12 files:   0% 0/12 [00:00<?, ?it/s]
Fetching 12 files:   8% 1/12 [00:00<00:06,  1.59it/s]




Fetching 12 files:  17% 2/12 [00:00<00:03,  2.63it/s]












































Fetching 12 files: 100% 12/12 [00:11<00:00,  1.05it/s]

Loading pipeline components...:   0% 0/7 [00:00<?, ?it/s]
Loading pipeline components...:  29% 2/7 [00:00<00:00, 18.97it/s]`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id

# Training with prior preservation loss

In [ ]:
!accelerate launch diffusers/examples/research_projects/dreambooth_inpaint/train_dreambooth_inpaint.py \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-inpainting"  \
  --instance_data_dir=$train_data_path \
  --class_data_dir=$class_data_path \
  --output_dir=$checkpoints_path \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --instance_prompt="thirty meter resolution satellite image, localized wildfire in a broad environment" \
  --class_prompt="thirty meter resolution satellite image, localized wildfire in a broad environment" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --use_8bit_adam \
  --learning_rate=$lr \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=200 \
  --max_train_steps=$max_train_steps \
  --checkpointing_steps=1000

2023-06-28 11:14:40.935739: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-28 11:14:52.755212: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to

# Training text encoder

In [ ]:
!accelerate launch diffusers/examples/research_projects/dreambooth_inpaint/train_dreambooth_inpaint.py \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-inpainting"  \
  --train_text_encoder \
  --instance_data_dir=$train_data_path \
  --output_dir=$checkpoints_path \
  --instance_prompt="satellite imagery, wildfire, landsat-8" \
  --resolution=256 \
  --train_batch_size=2 \
  --gradient_accumulation_steps=2 \
  --learning_rate=$lr \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=$max_train_steps \
  --checkpointing_steps=3000

2023-10-04 20:17:48.625924: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Steps: 100% 2000/2000 [16:14<00:00,  2.07it/s, loss=0.00453, lr=1e-6] 
vae/diffusion_pytorch_model.safetensors not found

Fetching 12 files:   0% 0/12 [00:00<?, ?it/s]
Fetching 12 files:   8% 1/12 [00:00<00:04,  2.31it/s]




Fetching 12 files:  33% 4/12 [00:00<00:01,  6.83it/s]






























































































Fetching 12 files: 100% 12/12 [00:28<00:00,  2.34s/it]

Loading pipeline components...:   0% 0/7 [00:00<?, ?it/s]
Loading pipeline components...:  14% 1/7 [00:00<00:01,  4.07it/s]`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be 